# Open Data management & Cloud PLM project debugging notebook

In [1]:
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'PLM.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

## Create New Part

In [3]:
from plm.models import PartCategory, PurchaseOption, Part
purchase_option = PurchaseOption.objects.create()
category, created = PartCategory.objects.get_or_create(name='MEC')

#part = Part.objects.create(description='Test part', purchase_option=purchase_option, category=category)
part = Part(description='Test part', purchase_option=purchase_option, category=category)

# part = Part()
# part.create_new_item(description='Test part', purchase_option=purchase_option, category=category)
part.save()
print(part.internal_part_number)

HW-MEC45647-v01r01


## Populate the DB

In [4]:
from django.apps import apps

models = apps.get_models()

for model in models:
    model.objects.all().delete()

In [ ]:
from plm.models import PartCategory, DocumentCategory, PurchaseOption, Part, Manufacturer, PurchaseOption, Document, Container

# Populating PartCategory
PartCategory.objects.create(name='MEC', description='Mechanical Components')
PartCategory.objects.create(name='ELE', description='Electrical Components')

# Populating DocumentCategory
DocumentCategory.objects.create(name='DRW', description='Drawings')
DocumentCategory.objects.create(name='DS', description='Datasheets')

# Populating Manufacturer
Manufacturer.objects.create(name='Mfr1', contact_info='Contact Info 1')
Manufacturer.objects.create(name='Mfr2', contact_info='Contact Info 2')

# Populating Part
part1 = Part.objects.create(description='Part 1', category=PartCategory.objects.get(name='MEC'), repository_link='http://example.com/repo1')
part2 = Part.objects.create(description='Part 2', category=PartCategory.objects.get(name='ELE'), repository_link='http://example.com/repo2')
part3 = Part.objects.create(description='Part 3', category=PartCategory.objects.get(name='MEC'), repository_link='http://example.com/repo3')
part4 = Part.objects.create(description='Part 4', category=PartCategory.objects.get(name='MEC'), repository_link='http://example.com/repo4')

# Populating PurchaseOption
purchase_option1 = PurchaseOption.objects.create(part_ipn=part1, manufacturer=Manufacturer.objects.get(name='Mfr1'), manufacturer_code='CODE1', datasheet='http://example.com/datasheet1')
purchase_option2 = PurchaseOption.objects.create(part_ipn=part2, manufacturer=Manufacturer.objects.get(name='Mfr2'), manufacturer_code='CODE2', datasheet='http://example.com/datasheet2')

# Populating Document
document1 = Document.objects.create(document_number='DOC001', description='Document 1', category=DocumentCategory.objects.get(name='DRW'), item_number=1, link='http://example.com/doc1')
document2 = Document.objects.create(document_number='DOC002', description='Document 2', category=DocumentCategory.objects.get(name='DS'), item_number=2, link='http://example.com/doc2')

# Populating Container (assuming parts and documents are already created)
container1 = Container.objects.create(part_a=part1, part_b=part2, quantity=5)
container2 = Container.objects.create(part_a=part3, part_b=part2, quantity=3)
container3 = Container.objects.create(part_a=part3, part_b=part4, quantity=3)

## Test API

In [81]:
#API GET parts list
import requests
import json
response = requests.get("http://localhost:8000/api/parts/?page=1&page_size=100")
#!curl -s "http://localhost:8000/api/parts/?page=1&page_size=100"
data = response.json()
data['results'][-1]

{'id': 20,
 'internal_part_number': 'HW-MEC023-v01r01',
 'description': 'Resistor 2.2k Ohm',
 'released': False,
 'purchase_option': 1,
 'repository_link': None,
 'storage_code': None,
 'creation_date': '2024-11-17',
 'category': 1,
 'item_number': 23,
 'variant': '1.00',
 'revision': '1.00'}

In [83]:
import requests
import json

data = {
    "internal_part_number": "HW-MEC024-v01r01",
    "description": "Resistor 2.2k Ohm",
    "purchase_option": 1,
    "category": 1,
    "item_number": 24,
    "variant": 1.0,
    "revision": 1.0
}

response = requests.post("http://localhost:8000/api/parts/", json=data)

if response.status_code == 201:
    new_part = response.json()
    print(new_part)
else:
    error_message = response.json()["detail"]
    print("Error:", error_message)

{'id': 21, 'internal_part_number': 'HW-MEC024-v01r01', 'description': 'Resistor 2.2k Ohm', 'released': False, 'purchase_option': 1, 'repository_link': None, 'storage_code': None, 'creation_date': '2024-11-17', 'category': 1, 'item_number': 24, 'variant': '1.00', 'revision': '1.00'}


In [ ]:
import requests
import json

base_url = "http://localhost:8000/api/"

# Create a new part
data = {
    "description": "Resistor 2.2k Ohm",
    "purchase_option": 1,
    "category": 1,
}
response = requests.post(base_url + "parts/create_new/", json=data)

# if response.status_code == 201:
#     new_part = response.json()
#     print("New part created:", new_part)
# else:
#     error_message = response.json().get("detail", "Unknown error")
#     print("Error creating part:", error_message)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [4]:
import requests
import json

data = {'document_id': 1}

response = requests.post("http://localhost:8000/api/parts/1/remove_document/", json=data)

if response.status_code == 201:
    new_part = response.json()
    print(new_part)
else:
    error_message = response.json()["detail"]
    print("Error:", error_message)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [4]:
import requests
import json

data = {"part_a": 1,"part_b": 2,"quantity": 5}

response = requests.post("http://localhost:8000/api/containers/", json=data)

if response.status_code == 201:
    new_part = response.json()
    print(new_part)
else:
    error_message = response.json()["detail"]
    print("Error:", error_message)

KeyError: 'detail'